In [1]:
import numpy as np

In [2]:
test_data = np.load('lab4_data/test_sentences.npy', allow_pickle=True)
test_labels = np.load('lab4_data/test_labels.npy', allow_pickle=True)

train_data = np.load('lab4_data/training_sentences.npy', allow_pickle=True)
train_labels = np.load('lab4_data/training_labels.npy', allow_pickle=True)

In [3]:
def normalize_data(train_data, test_data, type=None):
    if type == 'standard':
        medie_train = np.mean(train_data, axis=0)
        deviatie_train = np.std(train_data, axis=0)
        norm_train_data = (train_data - medie_train) / deviatie_train

        # Intotdeauna antrenam pe train si testam pe test !!!
        norm_test_data = test_data
    elif type == 'l1':
        norm_train = np.sum(np.abs(train_data), axis=1, keepdis=True) + 10 ** -8
        norm_train_data = train_data / norm_train
        norm_test = np.sum(np.abs(test_data), axis=1, keepdims=True) + 10 ** -8
        norm_test_data = test_data / norm_test
    elif type == 'l2':
        norm_train = np.sqrt(np.sum(train_data ** 2, axis=1, keepdims=True)) + 10 ** -8
        norm_train_data = train_data / norm_train
        norm_test = np.sum(np.abs(test_data ** 2), axis=1, keepdims=True) + 10 ** -8
        norm_test_data = test_data / norm_test
    else:
        raise ValueError('Invalid normalization type')

    return norm_train_data, norm_test_data

In [4]:
class BagOfWords:
    def __init__(self):
        self.vocabulary = {}
        self.voc_len = 0
        self.words = []

    def build_vocabulary(self, data):
        for sentence in data:
            for word in sentence:
                if word not in self.vocabulary:
                    self.vocabulary[word] = len(self.vocabulary)
                    self.words.append(word)
            self.voc_len = len(self.vocabulary)

    def get_features(self, data):
        features = np.zeros((len(data), self.voc_len))

        for id_sen, sentence in enumerate(data):
            for word in sentence:
                if word in self.vocabulary:
                    features[id_sen, self.vocabulary[word]] += 1
        return features

In [5]:
bag_of_words = BagOfWords()
bag_of_words.build_vocabulary(train_data)

In [6]:
train_features = bag_of_words.get_features(train_data)

test_features = bag_of_words.get_features(test_data)
train_features_norm, test_features_norm = normalize_data(train_features, test_features, type='l2')

In [ ]:
from sklearn import svm
svm_model = svm.SVC(C=1, kernel='linear')
svm_model.fit(train_features_norm, train_labels)
